### Twin SVM using Python
#### Author: Adel Ahmadi

Lets import libraries at first!

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

Lets load our dataset and create X and Y.

In [2]:
# Loading Dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paperxd/cleaned-life-expectancy-dataset")
filename = path + '/Cleaned-Life-Exp.csv'
data = pd.read_csv(filename)
data = data.drop(columns=['Country'])

# Creating X
X = data.drop(columns=['Status'])

# Convert labels to +-1 and Creating Y
# Binning countinues life expectancy
bins = [-4,-1.5,0.5,3]
labels = ['Low','Medium','High']
data['lifeexp_category'] = pd.cut(data['Life expectancy'], bins = bins, labels = labels)
y = data['lifeexp_category']
y = np.where(y == 'High', 1, -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

We create a class named TwinSVM for fitting the model without any extra libraries. also we have a predict function for predicting new datas with this.

In [3]:
class TwinSVM:
    def __init__(self, penalty=1.0):
        self.penalty = penalty  

    def fit(self, data, labels):
        group1 = data[labels == 1]
        group2 = data[labels == -1]
        
        # Adding bias term to each group
        augmented1 = np.hstack((group1, np.ones((group1.shape[0], 1))))
        augmented2 = np.hstack((group2, np.ones((group2.shape[0], 1))))
        
        # First plane's calculation
        A1 = np.dot(augmented1.T, augmented1) + self.penalty * np.identity(augmented1.shape[1])
        b1 = -np.dot(augmented2.T, np.ones(augmented2.shape[0]))
        self.hyperplane1 = np.linalg.solve(A1, b1)
        
        # Second plane's calculation
        A2 = np.dot(augmented2.T, augmented2) + self.penalty * np.identity(augmented2.shape[1])
        b2 = -np.dot(augmented1.T, np.ones(augmented1.shape[0]))
        self.hyperplane2 = np.linalg.solve(A2, b2)

    def predict(self, data):
        d1 = np.dot(data, self.hyperplane1[:-1]) + self.hyperplane1[-1]
        d2 = np.dot(data, self.hyperplane2[:-1]) + self.hyperplane2[-1]
        
        # Choose class based on closest distance
        return np.where(np.abs(d1) < np.abs(d2), 1, -1)

Lets train our model!

In [4]:
model = TwinSVM()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8095238095238095
